In [3]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from citipy import citipy
#import api key from file
from apikey import googlekey

# Configure gmaps
gmaps.configure(api_key=googlekey)

In [4]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps



Enabling notebook extension jupyter-js-widgets/extension...
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\Scripts\jupyter-nbextension-script.py", line 10, in <module>
    sys.exit(main())
  File "C:\ProgramData\Anaconda3\lib\site-packages\jupyter_core\application.py", line 254, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\notebook\nbextensions.py", line 980, in start
    super().start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\jupyter_core\application.py", line 243, in start
    self.subapp.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\notebook\nbextensions.py", line 888, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "C:\ProgramData\Anaconda3\lib\site-packages\notebook\nbextensions.py", line 861

In [5]:
csvpath = os.path.join("..","Data","forecast_data.csv")

# read in WeatherPy CSV file
citydata = pd.read_csv(csvpath)
display(citydata)

,Unnamed: 0,City Name,Latitude,Longitude,Country,UTC +/- (hours),Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,-54.8000,-68.3000,AR,-3.0,42.13,84,92,3.33
1,1,Kloulklubed,7.0419,134.2556,PW,9.0,84.13,85,94,21.12
2,2,Butaritari,3.0707,172.7902,KI,12.0,83.55,74,99,14.90
3,3,Syedove,47.0768,38.1560,UA,3.0,65.91,73,97,10.16
4,4,Mahébourg,-20.4081,57.7000,MU,4.0,79.83,78,40,10.29
...,...,...,...,...,...,...,...,...,...,...
562,562,General Roca,-39.0333,-67.5833,AR,-3.0,65.61,52,100,19.55
563,563,Redding,40.5865,-122.3917,US,-7.0,81.77,28,7,6.53
564,564,Kholm,59.2667,32.8500,RU,3.0,50.34,43,81,5.59
565,565,Penzance,50.1186,-5.5371,GB,1.0,50.25,60,3,12.95


In [6]:
citydata.Latitude.max()

78.2186

In [7]:
# plot heatmap of city humidity on google maps
citylocs = citydata[["Latitude","Longitude"]]
humidity = citydata["Humidity"]

fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(citylocs,weights=humidity,
                                     dissipating=False,max_intensity =100,
                                     point_radius=1)
fig.add_layer(humidity_layer)
display(fig)

Figure(layout=FigureLayout(height='420px'))

In [13]:
# filter city data to find cities with ideal weather conditions
idealvacation = citydata[(citydata["Humidity"] < 40)
                         & (citydata["Max Temperature"]< 75)
                         & (citydata["Max Temperature"]> 60)
                         & (citydata["Cloudiness"] < 50)
                         & (citydata["Wind Speed"]>10)
                         & (citydata["Wind Speed"]<20)
                         & (citydata["Country"] == "US") ]

display(idealvacation)

,Unnamed: 0,City Name,Latitude,Longitude,Country,UTC +/- (hours),Max Temperature,Humidity,Cloudiness,Wind Speed
20,20,Amberley,39.2048,-84.4280,US,-4.0,70.36,34,1,11.81
109,109,North Myrtle Beach,33.8160,-78.6800,US,-4.0,71.20,25,0,16.75
253,253,Charlottesville,38.0293,-78.4767,US,-4.0,69.94,20,8,12.59
368,368,Wyndham,37.6985,-77.6125,US,-4.0,68.09,20,5,13.51
373,373,Fairview,40.8126,-73.9990,US,-4.0,63.03,38,3,18.86
388,388,Staunton,38.1496,-79.0717,US,-4.0,66.67,29,7,17.87


In [10]:
#for each city in citylist
    # Call google places API to find the first hotel result within 5000 meters of each city
    #save hotel name, city, and country data to an existing list or dictionary

# Specify search parameters
target_coordinates = "43.6187102, -116.2146068" # Note: these must be strings
target_search = "Hotel"
target_radius = 8000 # Note: this is in meters
target_type = "lodging" # for more options here, see https://developers.google.com/places/web-service/supported_types

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": googlekey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" # nearbysearch, textSearch, PlaceSearch are all optioins here; verify syntax!

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

In [12]:
pprint(response)

{'html_attributions': [],
 'next_page_token': 'ATtYBwKKs7a9himyEfGQXUuQEEkiwVv6yaauvT0mJnVTS-0D9EFNwkDchUk9MngdXxxwzyn27-9oDbXmv0HicXBtl5S3XyBUw7JTQTA3RrBFReJ-66msMAPRl0_45ksTPIT1GAdqE-maGfHS-V-Mvj3vX9ezXFV-TqfQ3cqvpTJDodPzjEMNtutEhM0Ti_gQ99fEzfqQheogMlcr7cNOLBNeFW5ie65vR1HByTWk17n1derhEmU47iXem8nTxkg1gZqwYvju_fBkgZrojJcbBWiVciaZSNH4w1mLeFrHXmQkjmsqX_FYqm2UuTuB0zDYd3wt67Xf_5sMqt27lwjvo2UbGpwf2sol108rKyUgiNMBesKjS-XGnGbSbM2_wl5JElBFke8XEBddh-BUE7CLh_VciXhzo0Jn1j9X7uSlKCi9p2iGFpIfV3lkSmrr8h7YnbHv',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 43.6139805, 'lng': -116.2037792},
                           'viewport': {'northeast': {'lat': 43.61524747989272,
                                                      'lng': -116.2022788701073},
                                        'southwest': {'lat': 43.61254782010728,
                                                      'lng': -116.2049785298927}}},
              'icon': 'https://maps.gstatic

In [ ]:
#plot these hotels on the humidity heatmap with pins containing hotel name, city, and country